In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging

from lmkgroup_ds_utils.db.connector import DB


In [ ]:

class args:
    company = 'GL'
    db_pw = ''
    local = True
    write_to_db = False
    save_output_locally = True

company = args.company
local = args.local
save_output_locally = args.save_output_locally
db_pw = args.db_pw


In [ ]:
from paths import CONFIG_DIR, SQL_DIR
from utils import fetch_data_from_sql, read_yaml
db_configs = read_yaml(
    file_name="db",
    directory=CONFIG_DIR
)

read_db_name = db_configs["read"]["db_name"]
read_db_env = db_configs["read"]["env"]
read_db_settings_path = db_configs["read"]["db_settings_path"]

In [ ]:
logging.info("Connecting to db...")
read_db = DB(
    local=local,
    db_name=read_db_name,
    db_settings_path=read_db_settings_path,
    password=db_pw,
    env=read_db_env
)


In [ ]:
df_order_history = fetch_data_from_sql(
    read_db=read_db,
    sql_name="orders",
    directory=SQL_DIR,
    min_year=2017
)

In [ ]:
df_order_history = df_order_history.sort_values(
    by=["year", "week"]
)

df_order_history["num_dishes_orders"] = df_order_history["num_dishes_orders"].fillna(0)
df_order_history = df_order_history[df_order_history['num_total_orders']>=50]

In [ ]:
import plotly.express as px

fig = px.line(df_order_history, x="week", y="num_total_orders", color='year')
fig.show()

In [ ]:
import plotly.express as px

fig = px.line(df_order_history, x="week", y="num_mealbox_orders", color='year')
fig.show()

In [ ]:
import plotly.express as px

fig = px.line(df_order_history, x="week", y="num_dishes_orders", color='year')
fig.show()

In [ ]:
from orders_forecasting.helpers import create_date_from_year_week
df_order_history = create_date_from_year_week(
    df=df_order_history,
    day_of_week=1,
    date_column_name="first_date_of_week"
)

In [ ]:
import plotly.express as px

fig = px.line(df_order_history, x="first_date_of_week", y="num_dishes_orders", color='year')
fig.show()

In [ ]:
import plotly.express as px

fig = px.line(df_order_history, x="first_date_of_week", y="num_mealbox_orders", color='year')
fig.show()

In [ ]:
import plotly.express as px

fig = px.line(df_order_history, x="first_date_of_week", y="num_total_orders", color='year')
fig.show()

In [ ]:
df = df_order_history[["year", "week", "num_total_orders", "first_date_of_week"]]
df_train = df[df["first_date_of_week"]<='2023-09-01']
df_test = df[df["first_date_of_week"]>='2023-09-01']
X_train = df_train.drop(columns=["num_total_orders", "first_date_of_week"]).values
y_train = df_train["num_total_orders"]

X_test = df_test.drop(columns=["num_total_orders", "first_date_of_week"]).values
y_test = df_test["num_total_orders"]

In [ ]:
automl.best_loss

In [ ]:
y_pred = automl.predict(X_test)


In [ ]:
y_pred = automl.predict(X_test)

df_test['pred'] = y_pred
df_test["mape"] = abs(y_test - y_pred)/y_test

In [ ]:
df_test

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_train["first_date_of_week"], y=df_train["num_total_orders"],
                    mode='lines',
                    name='lines'))
fig.add_trace(go.Scatter(x=df_test["first_date_of_week"], y=df_test["num_total_orders"],
                    mode='lines+markers',
                    name='actual'))

fig.add_trace(go.Scatter(x=df_test["first_date_of_week"], y=df_test["pred"],
                    mode='lines+markers',
                    name='pred'))
fig.show()